# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
from datetime import timedelta
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Format today and yesterday into strings for filename creation
format = "%m-%d-%Y"
now = datetime.now()
today = now.strftime(format)
yesterday = (now - timedelta(days=1)).strftime(format)

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    jh_live_date = today
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    jh_live_date = yesterday
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
print(jh_live_date)

01-26-2023


In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2023-01-27 04:20:58,33.93911,67.709953,208324,7872,NaN,NaN,Afghanistan,535.147388,3.778729
1,NaN,NaN,NaN,Albania,2023-01-27 04:20:58,41.15330,20.168300,334135,3596,NaN,NaN,Albania,11610.779067,1.076212
2,NaN,NaN,NaN,Algeria,2023-01-27 04:20:58,28.03390,1.659600,271364,6881,NaN,NaN,Algeria,618.831347,2.535708
3,NaN,NaN,NaN,Andorra,2023-01-27 04:20:58,42.50630,1.521800,47820,165,NaN,NaN,Andorra,61890.894972,0.345044
4,NaN,NaN,NaN,Angola,2023-01-27 04:20:58,-11.20270,17.873900,105184,1931,NaN,NaN,Angola,320.036336,1.835831


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,1/17/23,1/18/23,1/19/23,1/20/23,1/21/23,1/22/23,1/23/23,1/24/23,1/25/23,1/26/23
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,208009,208034,208062,208084,208084,208084,208097,208289,208324,208324
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,334064,334084,334084,334084,334097,334101,334101,334113,334124,334135
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,271292,271296,271307,271316,271328,271335,271346,271354,271360,271364


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,1/17/23,1/18/23,1/19/23,1/20/23,1/21/23,1/22/23,1/23/23,1/24/23,1/25/23,1/26/23
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,7859,7860,7864,7864,7864,7864,7870,7871,7872,7872
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,3596,3596,3596,3596,3596,3596,3596,3596,3596,3596
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,6881,6881,6881,6881,6881,6881,6881,6881,6881,6881


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,1/17/23,1/18/23,1/19/23,1/20/23,1/21/23,1/22/23,1/23/23,1/24/23,1/25/23,1/26/23
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,1/17/23,1/18/23,1/19/23,1/20/23,1/21/23,1/22/23,1/23/23,1/24/23,1/25/23,1/26/23
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,19318,19389,19389,19389,19389,19389,19389,19389,19471,19471
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,68518,68764,68764,68764,68764,68764,68764,68764,68983,68983
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,7188,7258,7258,7258,7258,7258,7258,7258,7299,7299


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,1/17/23,1/18/23,1/19/23,1/20/23,1/21/23,1/22/23,1/23/23,1/24/23,1/25/23,1/26/23
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,230,230,230,230,230,230,230,230,230,230
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,721,722,722,722,722,722,722,722,723,723
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,103,103,103,103,103,103,103,103,103,103


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
678,1001.0,Autauga,Alabama,US,2023-01-27 04:20:58,32.539527,-86.644082,19471,230,NaN,NaN,"Autauga, Alabama, US",34851.169701,1.181244
715,1075.0,Lamar,Alabama,US,2023-01-27 04:20:58,33.779950,-88.096680,4661,73,NaN,NaN,"Lamar, Alabama, US",33763.129301,1.566188
716,1077.0,Lauderdale,Alabama,US,2023-01-27 04:20:58,34.901719,-87.656247,29954,419,NaN,NaN,"Lauderdale, Alabama, US",32302.731616,1.398812


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,102259155,1107542,0.0
India,44682338,530738,0.0
France,39708282,165077,0.0


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,102259155,1107542,0.0,101151613.0,2023-01-27 04:20:58,37.934269,-91.444083
India,44682338,530738,0.0,44151600.0,2023-01-27 04:20:58,23.088275,81.806127
France,39708282,165077,0.0,39543205.0,2023-01-27 04:20:58,6.430808,-34.730285
Germany,37728155,165314,0.0,37562841.0,2023-01-27 04:20:58,51.571844,10.277881
Brazil,36768677,696603,0.0,36072074.0,2023-01-27 04:20:58,-12.669522,-48.480493


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,11978785,100009,0.0
Texas,8351893,92559,0.0
Florida,7443954,84927,0.0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,11978785,100009,0.0,11878776.0,2023-01-27 04:20:58,37.843962,-120.728594
Texas,8351893,92559,0.0,8259334.0,2023-01-27 04:20:58,31.660643,-98.653069
Florida,7443954,84927,0.0,7359027.0,2023-01-27 04:20:58,28.940755,-82.700744
New York,6697789,76247,0.0,6621542.0,2023-01-27 04:20:58,42.544151,-75.474183
Illinois,4019768,41088,0.0,3978680.0,2023-01-27 04:20:58,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,3672125,35212,0.0
Florida,Miami-Dade,1524998,12049,0.0
Illinois,Cook,1507271,15161,0.0
Arizona,Maricopa,1495588,18665,0.0
Texas,Harris,1258801,11529,0.0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,3672125,35212,0.0,3636913.0,2023-01-27 04:20:58,34.308284,-118.228241,6037.0
Florida,Miami-Dade,1524998,12049,0.0,1512949.0,2023-01-27 04:20:58,25.611236,-80.551706,12086.0
Illinois,Cook,1507271,15161,0.0,1492110.0,2023-01-27 04:20:58,41.841448,-87.816588,17031.0
Arizona,Maricopa,1495588,18665,0.0,1476923.0,2023-01-27 04:20:58,33.348359,-112.491815,4013.0
Texas,Harris,1258801,11529,0.0,1247272.0,2023-01-27 04:20:58,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2023-01-24,208289,334113,271354,47820,105184,11,9106,10032709,446008,11274262,...,1031581,250443,12014,551297,11526339,703228,535,11945,339502,261606
2023-01-25,208324,334124,271360,47820,105184,11,9106,10032709,446008,11274262,...,1031581,250463,12014,551351,11526348,703228,535,11945,339743,261606
2023-01-26,208324,334135,271364,47820,105184,11,9106,10032709,446008,11292729,...,1031581,250463,12014,551389,11526365,703228,535,11945,339949,261606


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2023-01-24,7871,3596,6881,165,1931,0,146,130394,8717,18092,...,7601,1637,14,5840,43186,5708,0,2159,4041,5652
2023-01-25,7872,3596,6881,165,1931,0,146,130394,8717,18092,...,7601,1637,14,5840,43186,5708,0,2159,4041,5652
2023-01-26,7872,3596,6881,165,1931,0,146,130394,8717,18596,...,7601,1637,14,5843,43186,5708,0,2159,4043,5652


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2023-01-24,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2023-01-25,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2023-01-26,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,1/17/23,1/18/23,1/19/23,1/20/23,1/21/23,1/22/23,1/23/23,1/24/23,1/25/23,1/26/23
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,19318,19389,19389,19389,19389,19389,19389,19389,19471,19471
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,4626,4635,4635,4635,4635,4635,4635,4635,4661,4661
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,29636,29801,29801,29801,29801,29801,29801,29801,29954,29954
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,9210,9243,9243,9243,9243,9243,9243,9243,9289,9289
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,46236,46420,46420,46420,46420,46420,46420,46420,46651,46651


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2023-01-24,1602891,303575,8317,2394646,994837,11971887,1745313,963981,325083,49,...,2468351,8331513,1080067,149687,24385,2251042,1899401,632661,1979126,183774
2023-01-25,1610535,303575,8317,2398200,994837,11973116,1746349,964476,325872,49,...,2468351,8351255,1080067,150190,24425,2251042,1904198,632957,1979913,183774
2023-01-26,1610535,303575,8317,2398200,995505,11978785,1746349,964903,325872,49,...,2468351,8351893,1081346,150190,24460,2251042,1904183,633282,1980617,183774


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2023-01-24       19389   68764    7258  7889  18130    2956   6347   40377   
2023-01-25       19471   68983    7299  7919  18255    2970   6410   40586   
2023-01-26       19471   68983    7299  7919  18255    2970   6410   40586   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2023-01-24        10599     6628  ...    7648   2255     9789     2305   
2023-01-25        10656     6668  ...    7648   2255     9789     2305   
2023-01-26        10656     6668  ...    7648   2255     9789     2305   

Province_State                                                     
Admin2         Sweetwater  Teton Uinta Unassigned Washakie Weston  
2023-01-24          12454  12082  6353          0     2737   1885  
2023-01-25          12454  12082  6353          0     2737   1885  
2023-01-26          12454  12082  6353          0     2737   1885  

[3 rows x 3336 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2023-01-24,20846,1473,34,32631,12800,99773,13995,11942,3243,0,...,28883,92372,5234,884,129,23088,15312,7808,16046,1977
2023-01-25,20870,1473,34,32775,12800,99817,14024,11942,3254,0,...,28883,92552,5234,889,129,23088,15380,7808,16070,1977
2023-01-26,20870,1473,34,32775,12825,100009,14024,11941,3254,0,...,28883,92559,5253,889,129,23088,15381,7822,16092,1977


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2023-01-24         230     722     103  109    261      54    130     672   
2023-01-25         230     723     103  109    261      54    131     674   
2023-01-26         230     723     103  109    261      54    131     674   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2023-01-24          170       89  ...     152     44       87       28   
2023-01-25          170       89  ...     152     44       87       28   
2023-01-26          170       89  ...     152     44       87       28   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2023-01-24            137    16    43          0       47     22  
2023-01-25            137    16    43          0       47     22  
2023-01-26            137    16    43          0       47     22  

[3 rows x 3336 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change().round(7)
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2023-01-24,0.000923,0.000036,0.000029,0.0,0.000847,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.00006,0.0,0.000004,6.000000e-07,0.0,0.0,0.0,0.001522,0.00625
2023-01-25,0.000168,0.000033,0.000022,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.00008,0.0,0.000098,8.000000e-07,0.0,0.0,0.0,0.000710,0.00000
2023-01-26,0.000000,0.000033,0.000015,0.0,0.000000,0.0,0.0,0.0,0.0,0.001638,...,0.0,0.00000,0.0,0.000069,1.500000e-06,0.0,0.0,0.0,0.000606,0.00000


In [60]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [61]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change().round(7)
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2023-01-24,0.000127,0.0,0.0,0.0,0.000518,NaN,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,NaN,0.0,0.000495,0.002661
2023-01-25,0.000127,0.0,0.0,0.0,0.000000,NaN,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,NaN,0.0,0.000000,0.000000
2023-01-26,0.000000,0.0,0.0,0.0,0.000000,NaN,0.0,0.0,0.0,0.027858,...,0.0,0.0,0.0,0.000514,0.0,0.0,NaN,0.0,0.000495,0.000000


In [62]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [63]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change().round(7)
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2023-01-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [65]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change().round(7)
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2023-01-24,0.000000,0.002159,0.0,0.000000,0.000530,0.001221,0.000000,0.000680,0.000000,0.0,...,0.0,0.000137,0.000000,0.00000,0.000944,0.004736,0.000000,0.000395,0.000359,0.001024
2023-01-25,0.004769,0.000000,0.0,0.001484,0.000000,0.000103,0.000594,0.000513,0.002427,0.0,...,0.0,0.002370,0.000000,0.00336,0.001640,0.000000,0.002525,0.000468,0.000398,0.000000
2023-01-26,0.000000,0.000000,0.0,0.000000,0.000672,0.000474,0.000000,0.000443,0.000000,0.0,...,0.0,0.000076,0.001184,0.00000,0.001433,0.000000,-0.000008,0.000513,0.000356,0.000000


In [66]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [67]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change().round(7)
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2023-01-24      0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2023-01-25      0.004229  0.003185  0.005649  0.003803  0.006895  0.004736   
2023-01-26      0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2023-01-24      0.000000  0.000000  0.000000  0.000000  ...  0.000654   
2023-01-25      0.009926  0.005176  0.005378  0.006035  ...  0.000000   
2023-01-26      0.000000  0.000000  0.000000  0.000000  ...  0.000000   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2023-01-24      0.000444  0.001535  0.000868   0.000964  0.001409  0.001103   
2023-01-25      0.000000  0.000000  0.000000   0.000000  0.000000  0.000000   
2023-01-26      0.000000  0.000000  0.000000   0.000000  0.000000  0.000000   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2023-01-24            NaN  0.001464  0.000531  
2023-01-25            NaN  0.000000  0.000000  
2023-01-26            NaN  0.000000  0.000000  

[3 rows x 3336 columns]

In [68]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [69]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change().round(7)
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2023-01-24,0.000000,0.012371,0.0,0.000000,0.000938,0.000883,0.000000,0.003108,0.000000,NaN,...,0.0,0.000032,0.00000,0.000000,0.0,0.005487,0.000000,0.000000,0.000873,0.003553
2023-01-25,0.001151,0.000000,0.0,0.004413,0.000000,0.000441,0.002072,0.000000,0.003392,NaN,...,0.0,0.001949,0.00000,0.005656,0.0,0.000000,0.004441,0.000000,0.001496,0.000000
2023-01-26,0.000000,0.000000,0.0,0.000000,0.001953,0.001924,0.000000,-0.000084,0.000000,NaN,...,0.0,0.000076,0.00363,0.000000,0.0,0.000000,0.000065,0.001793,0.001369,0.000000


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change().round(7)
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                  \
Admin2         Autauga   Baldwin Barbour Bibb Blount Bullock    Butler   
2023-01-24         0.0  0.000000     0.0  0.0    0.0     0.0  0.000000   
2023-01-25         0.0  0.001385     0.0  0.0    0.0     0.0  0.007692   
2023-01-26         0.0  0.000000     0.0  0.0    0.0     0.0  0.000000   

Province_State                              ... Wyoming                  \
Admin2           Calhoun Chambers Cherokee  ...    Park Platte Sheridan   
2023-01-24      0.000000      0.0      0.0  ...     0.0    0.0      0.0   
2023-01-25      0.002976      0.0      0.0  ...     0.0    0.0      0.0   
2023-01-26      0.000000      0.0      0.0  ...     0.0    0.0      0.0   

Province_State                                                             
Admin2         Sublette Sweetwater Teton Uinta Unassigned Washakie Weston  
2023-01-24          0.0        0.0   0.0   0.0        NaN      0.0    0.0  
2023-01-25          0.0        0.0   0.0   0.0        NaN      0.0    0.0  
2023-01-26          0.0        0.0   0.0   0.0        NaN      0.0    0.0  

[3 rows x 3336 columns]

In [72]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [73]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean().round(7)
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2023-01-24,0.000485,0.000023,0.000033,0.000013,0.000423,0.0,0.0,0.000219,1.000000e-07,0.000050,...,0.000497,0.000105,0.0,0.000031,7.000000e-07,0.0,0.0,0.0,0.000874,0.003125
2023-01-25,0.000326,0.000028,0.000027,0.000006,0.000212,0.0,0.0,0.000109,1.000000e-07,0.000025,...,0.000249,0.000092,0.0,0.000064,8.000000e-07,0.0,0.0,0.0,0.000792,0.001563
2023-01-26,0.000163,0.000030,0.000021,0.000003,0.000106,0.0,0.0,0.000055,0.000000e+00,0.000831,...,0.000124,0.000046,0.0,0.000067,1.100000e-06,0.0,0.0,0.0,0.000699,0.000781


In [74]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [75]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean().round(7)
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2023-01-24,0.000265,0.0,0.0,0.0,0.000259,NaN,0.0,0.000109,2.000000e-07,0.000396,...,0.000134,0.0,0.0,0.000008,0.0,0.0,NaN,0.0,0.000272,0.001331
2023-01-25,0.000196,0.0,0.0,0.0,0.000130,NaN,0.0,0.000054,1.000000e-07,0.000198,...,0.000067,0.0,0.0,0.000004,0.0,0.0,NaN,0.0,0.000136,0.000665
2023-01-26,0.000098,0.0,0.0,0.0,0.000065,NaN,0.0,0.000027,1.000000e-07,0.014028,...,0.000034,0.0,0.0,0.000259,0.0,0.0,NaN,0.0,0.000316,0.000333


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [77]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean().round(7)
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antarctica,Antigua and Barbuda,Argentina,Armenia,Australia,...,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Winter Olympics 2022,Yemen,Zambia,Zimbabwe
2023-01-24,-0.5,-0.499907,-0.496523,-0.498569,-0.493288,NaN,-0.499171,-0.498809,-0.499661,-0.499402,...,-0.499758,-0.496783,-0.5,-0.498237,-0.453466,-0.499858,NaN,-0.497938,-0.498727,-0.489078
2023-01-25,-0.5,-0.499907,-0.496523,-0.498569,-0.493288,NaN,-0.499171,-0.498809,-0.499661,-0.499402,...,-0.499758,-0.496783,-0.5,-0.498237,-0.453466,-0.499858,NaN,-0.497938,-0.498727,-0.489078
2023-01-26,-0.5,-0.499907,-0.496523,-0.498569,-0.493288,NaN,-0.499171,-0.498809,-0.499661,-0.499402,...,-0.499758,-0.496783,-0.5,-0.498237,-0.453466,-0.499858,NaN,-0.497938,-0.498727,-0.489078


In [78]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [79]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean().round(7)
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2023-01-24,0.000033,0.001088,0.000030,0.000009,0.000362,0.000630,0.000204,0.000496,0.000046,0.0,...,0.000049,0.000122,0.000016,0.000021,0.001547,0.002387,0.000019,0.000468,0.000373,0.000516
2023-01-25,0.002401,0.000544,0.000015,0.000747,0.000181,0.000367,0.000399,0.000505,0.001236,0.0,...,0.000024,0.001246,0.000008,0.001690,0.001594,0.001193,0.001272,0.000468,0.000385,0.000258
2023-01-26,0.001201,0.000272,0.000008,0.000373,0.000426,0.000420,0.000199,0.000474,0.000618,0.0,...,0.000012,0.000661,0.000596,0.000845,0.001513,0.000597,0.000632,0.000491,0.000371,0.000129


In [80]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [81]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean().round(7)
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2023-01-24      0.000029  0.000028  0.000077  0.000034  0.000032  0.000046   
2023-01-25      0.002129  0.001607  0.002863  0.001919  0.003463  0.002391   
2023-01-26      0.001065  0.000803  0.001431  0.000959  0.001732  0.001195   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2023-01-24      0.000036  0.000036  0.000041  0.000046  ...  0.000332   
2023-01-25      0.004981  0.002606  0.002709  0.003040  ...  0.000166   
2023-01-26      0.002491  0.001303  0.001355  0.001520  ...  0.000083   

Province_State                                                              \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton    Uinta   
2023-01-24      0.000222  0.000771  0.000438   0.000484  0.000711  0.00056   
2023-01-25      0.000111  0.000385  0.000219   0.000242  0.000356  0.00028   
2023-01-26      0.000056  0.000193  0.000109   0.000121  0.000178  0.00014   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2023-01-24      -0.503937  0.000735  0.000272  
2023-01-25      -0.503937  0.000367  0.000136  
2023-01-26      -0.503937  0.000184  0.000068  

[3 rows x 3336 columns]

In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [83]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean().round(7)
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2023-01-24,0.000010,0.006186,0.0,0.000031,0.000535,0.000491,0.000204,0.001601,0.000112,NaN,...,0.000030,0.000043,0.000036,0.000034,0.000032,0.002768,0.000045,0.000250,0.000874,0.001783
2023-01-25,0.000581,0.003093,0.0,0.002222,0.000268,0.000466,0.001138,0.000800,0.001752,NaN,...,0.000015,0.000996,0.000018,0.002845,0.000016,0.001384,0.002243,0.000125,0.001185,0.000891
2023-01-26,0.000290,0.001546,0.0,0.001111,0.001110,0.001195,0.000569,0.000358,0.000876,NaN,...,0.000008,0.000536,0.001824,0.001422,0.000008,0.000692,0.001154,0.000959,0.001277,0.000446


In [84]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [85]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean().round(7)
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State Alabama                                                    \
Admin2         Autauga   Baldwin Barbour      Bibb        Blount Bullock   
2023-01-24         0.0  0.000011     0.0  0.000072  2.000000e-07     0.0   
2023-01-25         0.0  0.000698     0.0  0.000036  1.000000e-07     0.0   
2023-01-26         0.0  0.000349     0.0  0.000018  1.000000e-07     0.0   

Province_State                                        ...   Wyoming         \
Admin2            Butler   Calhoun Chambers Cherokee  ...      Park Platte   
2023-01-24      0.000000  0.000012      0.0      0.0  ...  0.000026    0.0   
2023-01-25      0.003846  0.001494      0.0      0.0  ...  0.000013    0.0   
2023-01-26      0.001923  0.000747      0.0      0.0  ...  0.000006    0.0   

Province_State                                                           \
Admin2              Sheridan Sublette Sweetwater Teton Uinta Unassigned   
2023-01-24      4.000000e-07      0.0   0.000029   0.0   0.0       -1.0   
2023-01-25      2.000000e-07      0.0   0.000014   0.0   0.0       -1.0   
2023-01-26      1.000000e-07      0.0   0.000007   0.0   0.0       -1.0   

Province_State                  
Admin2         Washakie Weston  
2023-01-24          0.0    0.0  
2023-01-25          0.0    0.0  
2023-01-26          0.0    0.0  

[3 rows x 3336 columns]

In [86]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [87]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

HTTPError: HTTP Error 403: Forbidden

In [ ]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [ ]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

In [ ]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [ ]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

In [ ]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

#### 